# Operation mode and memory management

https://www.tutorialspoint.com/operating_system/os_memory_management.htm

At here we only consider `segmentation` and `paging`

- `segmentation`
    - choose the memory size as I want
    - set up the address at segment register pointing at segment starting address or data structure called descriptor
- `paging`
    - size of the chunk memory is chosen
    - available when we set the `CR3`'s page directory(data structure) physical address

https://en.wikipedia.org/wiki/X86_memory_segmentation

## Real mode
- 1MB address area
- only support segmentation
- segment size is fixed to `64K`
- `segment starting address` is set up manually at segment register
- `segment starting address` is used as base address when we assess to code or memory

In real mode or V86 mode, the size of a segment can range from 1 byte up to 65,536 bytes (using 16-bit offsets).

The 16-bit segment selector in the segment register is interpreted as the most significant 16 bits of a linear 20-bit address, called a segment address, of which the remaining four least significant bits are all zeros. The segment address is always added to a 16-bit offset in the instruction to yield a linear address, which is the same as physical address in this mode. For instance, the segmented address 06EFh:1234h (here the suffix "h" means hexadecimal) has a segment selector of 06EFh, representing a segment address of 06EF0h, to which the offset is added, yielding the linear address 06EF0h + 1234h = 08124h.


<tt>&nbsp; </tt><tt style="background:#DED">0000 0110 1110 1111</tt><tt>&nbsp;0000</tt>
**Segment**
16 bits, shifted 4 bits left (or multiplied by 0x10)
<br>
<tt>+ &nbsp;&nbsp;&nbsp;&nbsp; </tt><tt style="background:#DDF">0001 0010 0011 0100</tt>
**Offset**
&nbsp;&nbsp;&nbsp;&nbsp; 16 bits
<tt>&nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</tt>

--------------------------------------------------------------

<tt>&nbsp; </tt><tt style="background:#FDF">0000 1000 0001 0010 0100</tt>
**Address**
&nbsp; 20 bits
<br>
![rm](https://upload.wikimedia.org/wikipedia/commons/thumb/d/db/Overlapping_realmode_segments.svg/300px-Overlapping_realmode_segments.svg.png)

## Protected mode
- Supported `segmentation` and `paging` both
- offers more function at `segmentation` compared to real mode `segmentation`
- segment register point different address (`segment starting address` -> `descriptor datastructure offset`)
- name changed (`segment register` -> `segment selector`)

**Descriptor** is a data structure stores the memory area information.

`Segment` information is recorded at `segment descriptor`

**segment descriptor**
- segment starting address
- memory size
- privilege
- type

Located at `GDT(Global Descriptor Table)`
- maximum **8192** descriptor could be saved
- **`GDTR`(Global Descriptor Table Register)** point `GDT`
    - Save physical address of `GDT` (`16 bit GDT size` field + `32 bit base address` field)

https://en.wikipedia.org/wiki/Segment_descriptor

|31  |  —  |24  |23   |22   |21   |20   |19  |  —  |16  |15   |14   |13   |12   |11   |10   |9    |8    |7   |  —  |0   |
|---:|:---:|:---|:---:|:---:|:---:|:---:|---:|:---:|:---|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|---:|:---:|:---|
||Base Address[31:24]||G |D/B|L |AVL| |Segment Limit[19:16]||P ||DPL  |1 |Type|C/E|R/W|A||Base Address[23:16]||
| |||||||||Base Address[15:0]|  ||||||||||                  Segment Limit[15:0]|

![ds](https://camo.githubusercontent.com/af61d948264b0fb8597e74b5e718c5608b4344aa/68747470733a2f2f692e696d6775722e636f6d2f58367052444b762e706e67)

This is actually pretty clear. Because we want to add `segmentation size` and `privilege` so we have to get more additional information. **In C we make additional data structure and save informations at there.** So we can think `CS` is pointing at `C structure` as a metaphor. The table is a step before going to the real address. This is smart.

**segmentation and paging**

![paging](https://commons.bmstu.wiki/images/8/82/P.jpg)

For easy understanding...

**P : program**
- program 1 : frame
- program 2 : frame

So each p means program and each program are mapped with each memory space.

![ptof](https://i.stack.imgur.com/jV3Wn.png)

### Multilevel paging

- 2 level : 4MB page
- 3 level : 4KB page 
    - logical address divided by 3 area -> `directory`, `table`, `offset` 

I think the following picture is actually 4 level 

![mp](https://media.geeksforgeeks.org/wp-content/uploads/20190608174704/multilevel.png)

This picture is same as `4 dimension list`! Using 4 pointer! ****!

1. \*`PTBR` + `level 1 offset` : first table PTE
2. \*(\*`PTBR` + `level 1 offset`) + **level 2 offset** : second table PTE
3. \*(\*(\*`PTBR` + `level 1 offset`) + **level 2 offset)** + **level 3 offset** : third table PTE
4. \*(\*(\*(\*`PTBR` + `level 1 offset`) + **level 2 offset)** + **level 3 offset**) + `offset` : physical address

***PTBR data structure***

**`physical address paging data structure` \*\*\*\*PTBR**

As `GDT` has `GDTR`, for **paging** it has `CR3`!

`CR3` control register points the page directory's starting address and it will be used calculating the `page directory entry address(location)`

- page directory entry size : 4 bytes
- page table entry size : 4 bytes

![page](https://wiki.osdev.org/images/9/9b/Page_table.png)

**logical address composition**
- directory offset : `10 bit`
- table offset : `10 bit`
- page offset : `12 bit`

So `page directory` and `page table` total entry is **1024** and `page offset` is **4096**.
- `12bit` is for `4KB`

Each `page directory entry` and `page table entry` points at the `page table starting address` and `page starting address`, respectively and **we get the physical address by adding the logical address offset to this starting address**.

![conversion](https://lh3.googleusercontent.com/proxy/LkAujzA47_8GGLsYQoyXQLKekJBhV5eKlsvExpLFmQiZNbl0Z42aQ7cgHOxYLprTr-rSUX5-4nGXasq_PcAnm288nSFZv1sX4MTrs9ca7ih_h43v3aU7TOK12FORn_7mOexkS5ajVcCOyJR4-09EQC8Xw6O7Vf1ckE0PgSJ-VPQT)

## IA-32e mode

1. Compatibility mode
2. 64 bit mode

`Compatibility mode` works exactly sames as protected mode. We are going to look at `64 bit mode`.

IA-32e mode's 64 bit mode is **64 bit** so we can use $2^{64}$(16 Exa Byte) numbers of address.

Exa Byte = $2^{60}$ = Giga Byte * $2^{30}$(1,073,741,824) = Tera Byte = $2^{20}$(1,048,576)<br>

It approximately **1,000,000,000** times bigger than the protected mode and has almost same memory management compared to protected mode. So we are going to look at the difference with the protected mode mostly.

**Difference**
1. Address space has been expanded
2. Two sub mode is supported and few function has added
    - Compatibility mode
    - 64 bit mode
    - for distinguish between mode, `Code segment descriptor` has `L field(bit 21)` added. When this field is 0 `compatibility mode` is on and when this field is 1 `64 bit mode` is on. This make us possible to run **32 bit code** without going back to `protected mode`
3. All segment has the following set up no matter with the segment descriptor set
    - base address is **0**
    - size is **64 bit**
    - ![64](https://nixhacker.com/content/images/2020/02/flat-model.jpg)
    - This happend because segment descriptor used at protected mode was built to only store 32 bit address, and by this to support the 64 bit address descriptor wasn't extended rather the value be fixed. So in `IA-32e` mode we have to build the OS considering that original address cannot divide the logical address to several segments.
    - If the previous `32 bit OS` distinguish `kernel` and `user` area by **segment base address** this is not valid at `IA-32e` mode so we have to consider paging or etc!
    


**paging mode**
- Different from `protected mode` the address area has been extended to **64 bit** so `PAE` function has been activated basically.
- 4KB page is 5 levels(3 levels at protected mode)
- 2MB page is 3 levels(2 levels at ")
- `page map level 4 table`(PML4, Page Map Level 4 Table) and `Page Directory Pointer Table`(PDPT) has been newly added 

![ie](https://i.stack.imgur.com/rvU8o.png)

Each table index has been diminished to **9 bit** so `entry number` is **512($2^9$)** now. We can see that `48 - 64 bit` is not used. So the total `48 bit` is used for address and we can use **256TB($2^{48}$)**. But this `48 bit` is not always used for 48 bit. It depends on the processor. If processor only support `40 bit` than the physical address is only supported till **1TB($2^{40}$)**.

More information about paging!
-> https://www.triplefault.io/2017/07/introduction-to-ia-32e-hardware-paging.html


![x86_64 paging entry](https://slideplayer.com/slide/9413283/28/images/37/Paging+in+64+bit+Mode+Physical+address+size+grows+to+M%3E32+bits.jpg)

- The `reserved area` is filled with 0!
- The `lower 32 bit` area of entry is same as `32 bit protecte mode`
- `EXB` field prevents commands executed at the page. This prevents command executed on the data area and makes OS more safer
    - If command must not be executed at data area because there are no program code at here.
    - malicious attacks make this happen
        - `Buffer-Overflow`
        - `Stack-Overflow`
        - Putting program code at `buffer` or `stack` and push data to this area more than it could store to force the code to be executed. The code stored at this moment is inside `buffer` or `stack` and this 2 area is included in the `data area`.
        - If `EXB` was set then **page fault exception** will occur when the command is executed and program stopped. So we could be able to defend the attack!


**Different from protected mode, paging is MUST at IA-32e mode!**